In [1]:
import os
import time
import pickle


In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import LeaveOneOut

In [4]:
feature_path = os.path.abspath('../../features/inception')
model_save_path = os.path.abspath('../../models/inception_svm')
leave_one_out_validation = False

In [5]:
#Storing the extracted features
non_umpire = np.load(f'{feature_path}/model1_inception_features_non_umpire.npy')
umpire = np.load(f'{feature_path}/model1_inception_features_umpire.npy')

In [6]:
X_data = np.append(non_umpire, umpire, axis=0)
labels = X_data[:, (X_data.shape[1]-1)]
train_data = X_data[:, 0:(X_data.shape[1]-1)]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size=0.2, random_state=10)

In [8]:
#Classifier using Linear SVM
clf = LinearSVC(C=10)
start_time = time.time()
clf = clf.fit(x_train, y_train)
predictions_tr = (clf.predict(x_test))

In [9]:
#10-fold Cross-Validation    
scores = cross_val_score(clf, x_train, y_train, cv=10)
test_acc = accuracy_score(y_test, predictions_tr)

In [10]:
print("Training Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("Test Accuracy: %0.4f" % test_acc)
print("--- %s seconds ---" % (time.time() - start_time))

Training Accuracy: 0.9409 (+/- 0.11)
Test Accuracy: 0.9667
--- 3.8959524631500244 seconds ---


In [14]:
#Leave One Out Validation
if leave_one_out_validation:
   loo_train_acc = []
   loo = LeaveOneOut()
   for train_index, test_index in loo.split(x_train):
      X_train, X_test = x_train[train_index], x_train[test_index]
      Y_train, Y_test = y_train[train_index], y_train[test_index]
      clf = clf.fit(X_train, Y_train)
      predictions = (clf.predict(X_test))
      loo_train_acc.append(accuracy_score(Y_test, predictions))

   loo_train_accuracy = np.asarray(loo_train_acc)
   print("LOO Accuracy: %0.4f" % loo_train_accuracy.mean())

In [15]:
#Save the model
pickle.dump(clf, open(f'{model_save_path}/model1_inception_svm.sav', 'wb'))